In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [2]:
train = pd.read_csv('C:/Users/s_csmscox/jupyterSave/fashion-mnist_train.csv')
test = pd.read_csv('C:/Users/s_csmscox/jupyterSave/fashion-mnist_test.csv')

train_label = train['label']
train.drop(['label'], axis=1, inplace=True)

test_label = test['label']
test.drop(['label'], axis=1, inplace=True)

# 정규화
scaler_train = MinMaxScaler()
scaler_train.fit(train)
x_data_train_norm = scaler_train.transform(train).astype(np.float32)

scaler_test = MinMaxScaler()
scaler_test.fit(test)
x_data_test_norm = scaler_test.transform(test).astype(np.float32)

# one hot encoding
### tensorflow 기능을 이용해서 one hot encoding을 생성
sess = tf.Session()
t_data_train_onehot = sess.run(tf.one_hot(train_label, depth=10))

In [3]:
# placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
T = tf.placeholder(shape=[None,10], dtype=tf.float32)

X_img = tf.reshape(X, [-1,28,28,1])

# Model
# Weight => filter(kernel)
W1 = tf.Variable(tf.random.normal([3,3,1,32]))
conv1 = tf.nn.conv2d(X_img,
                      W1,
                      strides=[1,1,1,1],
                      padding='SAME')
relu1 = tf.nn.relu(conv1)
layer1 = tf.nn.max_pool(relu1,
                         ksize=[1,2,2,1],
                         strides=[1,2,2,1],
                         padding='SAME') # (?, 14, 14, 32)
print(layer1.shape)

W2 = tf.Variable(tf.random.normal([3,3,32,64]))
conv2 = tf.nn.conv2d(layer1,
                      W2,
                      strides=[1,1,1,1],
                      padding='SAME')
relu2 = tf.nn.relu(conv2)
layer2 = tf.nn.max_pool(relu2,
                         ksize=[1,2,2,1],
                         strides=[1,2,2,1],
                         padding='SAME') # (?, 7, 7, 64)
print(layer2.shape)

flatten = tf.reshape(layer2, [-1, 7*7*64])

W3 = tf.get_variable('W3', shape=[7*7*64, 256],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random.normal([256]))
layer3 = tf.nn.relu(tf.matmul(flatten, W3) + b3)

W4 = tf.get_variable('W4', shape=[256,10],
                     initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random.normal([10]))

logit = tf.matmul(layer3, W4) + b4
H = tf.nn.softmax(logit)

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,
                                                                 labels=T))
# train
train = tf.train.AdamOptimizer(learning_rate=1e-1).minimize(loss)

(?, 14, 14, 32)
(?, 7, 7, 64)
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
num_of_epoch = 100

# Accuracy    
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(T,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

# K-Fold Cross Validation
cv = 3          # Fold의 수
results = []    
                
kf = KFold(n_splits=cv, shuffle=True) 

for training_idx, validation_idx in kf.split(x_data_train_norm):
    training_x = x_data_train_norm[training_idx] # Fancy indexing
    training_t = t_data_train_onehot[training_idx]
    
    val_x = x_data_train_norm[validation_idx]
    val_t = t_data_train_onehot[validation_idx]
    
    # run_train(sess,training_x,training_t)
    print('### 학습 시작 ###')
    sess.run(tf.global_variables_initializer())
    
    for step in range(num_of_epoch):
        _, loss_val = sess.run([train,loss],
                                   feed_dict={X: training_x, T: training_t})
    
        if step % 10 == 0:
            print('Loss : {}'.format(loss_val))
    
    acc = sess.run(accuracy, feed_dict={X:val_x, T:val_t})
    print('측정한 각각의 결과값 : {}'.format(acc))
    results.append(acc)


print('최종 K-Fold 교차검증을 사용한 Accuracy : {}'.format(np.mean(results)))

In [ ]:
result = sess.run(H, feed_dict={X: x_data_test_norm})
pred = []

for i in result:
    m = i.max()
    for j in range(10):
        if i[j] == m:
            pred.append(j)
            break

cnt = 0
for predict, answer in zip(pred, test_label):
    if predict == answer:
        cnt += 1

print("Accuracy : {}".format((cnt/len(pred))))